In [1]:
import base64
import boto3
import json


def understand_video(video_path, prompt):
    # Bedrock Runtime 클라이언트 생성 (us-east-1 리전 사용)
    client = boto3.client(
        "bedrock-runtime"
    )

    # 사용할 모델 ID 지정 (Nova Lite 비전 모델)
    MODEL_ID = "amazon.nova-lite-v1:0"

    # 비디오 파일 열기 및 Base64 인코딩
    with open(video_path, "rb") as video_file:
        binary_data = video_file.read()  # 비디오 파일을 바이너리로 읽기
        base_64_encoded_data = base64.b64encode(binary_data)  # base64 인코딩
        base64_string = base_64_encoded_data.decode("utf-8")  # 문자열로 디코딩

    # 시스템 메시지: 모델의 역할 정의
    system = [
        {
            "text": "당신은 예술 분야의 전문가입니다. 사용자 요청에 대해서 대답해주세요."
        }
    ]

    # 사용자 메시지: 비디오와 텍스트 프롬프트를 함께 전달
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "video": {
                        "format": "mp4",  # 비디오 포맷
                        "source": {"bytes": base64_string},  # base64 인코딩된 비디오
                    }
                },
                {
                    "text": prompt  # 텍스트 프롬프트
                },
            ],
        }
    ]

    # 추론 파라미터 설정
    inference_params = {
        "maxTokens": 1024,     # 최대 생성 토큰 수
        "topP": 0.1,          # top-p 샘플링
        "topK": 20,           # top-k 샘플링
        "temperature": 0.3    # 창의성 조절 (낮을수록 결정적)
    }

    # 모델 요청 바디 구성
    request_body = {
        "schemaVersion": "messages-v1",
        "messages": messages,
        "system": system,
        "inferenceConfig": inference_params,
    }

    # 모델 호출
    response = client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(request_body)
    )

    # 응답 본문 파싱
    model_response = json.loads(response["body"].read())

    # 사용량 정보 출력 (예: 사용한 토큰 수 등)
    print("\n[Usage]-------------------")
    print(json.dumps(model_response['usage'], indent=2))

    # 생성된 텍스트만 추출 및 출력
    content_text = model_response["output"]["message"]["content"][0]["text"]
    
    return content_text


In [2]:
# 실행
video_path = "sample/video.mp4"
prompt = "영상에 대해서 상세하게 설명해주세요."
result = understand_video(video_path, prompt)
print("\n[Response Content Text]-------------------")
print(result)


[Usage]-------------------
{
  "inputTokens": 2367,
  "outputTokens": 252,
  "totalTokens": 2619,
  "cacheReadInputTokenCount": 0,
  "cacheWriteInputTokenCount": 0
}

[Response Content Text]-------------------
해변의 모래 위에 큰 조개가 놓여 있습니다. 조개는 밝은 색의 껍질을 가지고 있으며, 그 껍질에는 여러 개의 나선형 패턴이 새겨져 있습니다. 조개 주변에는 작은 모래 입자가 흩어져 있습니다. 조개 옆에는 파도가 올라와 모래를 흩뿌리고 있습니다. 파도는 조개 위로 올라와 조금씩 뒤로 물러갑니다. 배경에는 푸른 바다가 보이며, 맑은 하늘 아래 파도가 부드럽게 움직이고 있습니다.
